In [1]:
# from pdbfixer import PDBFixer
from pathlib import Path
import mdtraj as md
import openmm as mm 
from openmm import app
from openmm import unit as U
import nglview as view

In [20]:
### Get out structure after minimization (Bonus trick)
current_reporter = app.pdbreporter.PDBReporter("../data/prototype1/structure_after_minimization.pdb",1)
current_reporter.report(simulation, simulation.context.getState(getPositions=True))

In [21]:
minimized = md.load_pdb("../data/prototype1/structure_after_minimization.pdb")
view.show_mdtraj(minimized)

NGLWidget()

In [22]:
n_steps = 10000
simulation.step(n_steps)

In [3]:
traj1 = md.load_pdb("../data/prototype1/run1.pdb")
view.show_mdtraj(traj1)

NGLWidget(max_frame=199)

In [33]:
base = md.load_pdb("../data/2efv.pdb")
print(list(base.topology.chains))
view.show_mdtraj(base)


[<mdtraj.core.topology.Chain object at 0x15f4a5a90>, <mdtraj.core.topology.Chain object at 0x15f4a5af0>]


NGLWidget()

In [3]:
base = md.load_pdb("../data/2efv_fixed.pdb")
view.show_mdtraj(base)

NGLWidget()

In [27]:
base.topology

<mdtraj.Topology with 1 chains, 92 residues, 762 atoms, 769 bonds at 0x160e9c820>

In [11]:
from OpenSMOG import OpenSMOG

In [13]:
pdb = OpenSMOG.pdbfile.PDBFile("../data/TrialSystem1/TrialSystem1.pdb")
gro = OpenSMOG.GromacsGroFile("../data/TrialSystem1/TrialSystem1.gro")
top = OpenSMOG.GromacsTopFile("../data/TrialSystem1/TrialSystem1.top")

In [14]:
top

In [3]:
from OpenSMOG import SBM
from pathlib import Path

    ****************************************************************************************    
      **** *** *** *** *** *** *** *** OpenSMOG (v1.1.1) *** *** *** *** *** *** *** ****       

           The OpenSMOG class is used to perform molecular dynamics simulations using           
                     Structure-Based Models (SBM) for biomolecular systems,                     
             and it allows for the simulation of a wide variety of potential forms.             
                      OpenSMOG uses force field files generated by SMOG 2.                      
                             OpenSMOG documentation is available at                             
                  https://opensmog.readthedocs.io and https://smog-server.org                   

                    OpenSMOG is described in: Oliveira and Contessoto et al,                    
              SMOG 2 and OpenSMOG: Extending the limits of structure-based models.              
                    Protein 

In [19]:
sbm_CA = SBM(name='2ci2', time_step=0.0005, collision_rate=1.0, r_cutoff=0.65, temperature=0.5)

In [24]:
sbm_CA.setup_openmm(platform='cpu')

In [42]:
sbm_CA

In [28]:
sbm_CA.saveFolder('../data/test2')

In [8]:

# sbm_xmlfile = "../data/TrialSystem1/TrialSystem1.xml"
# assert Path(sbm_grofile).exists()


In [2]:
import openmm as mm
from openmm import app

sbm_grofile = "../data/base/base.gro"
sbm_topfile = "../data/base/base.top"
top = app.GromacsTopFile(sbm_topfile)
pos = app.GromacsGroFile(sbm_grofile)

In [3]:
# top.topology.getPeriodicBoxVectors()
help(top.createSystem)
# for vec in pos.getPeriodicBoxVectors():
    # print (vec*2)

Help on method createSystem in module openmm.app.gromacstopfile:

createSystem(nonbondedMethod=NoCutoff, nonbondedCutoff=Quantity(value=1.0, unit=nanometer), constraints=None, rigidWater=True, implicitSolvent=None, soluteDielectric=1.0, solventDielectric=78.5, ewaldErrorTolerance=0.0005, removeCMMotion=True, hydrogenMass=None, switchDistance=None) method of openmm.app.gromacstopfile.GromacsTopFile instance
    Construct an OpenMM System representing the topology described by this
    top file.
    
    Parameters
    ----------
    nonbondedMethod : object=NoCutoff
        The method to use for nonbonded interactions.  Allowed values are
        NoCutoff, CutoffNonPeriodic, CutoffPeriodic, Ewald, PME, or LJPME.
    nonbondedCutoff : distance=1*nanometer
        The cutoff distance to use for nonbonded interactions
    constraints : object=None
        Specifies which bonds and angles should be implemented with
        constraints. Allowed values are None, HBonds, AllBonds, or HAngles.


In [14]:


system = top.createSystem(
    # nonbondedMethod=app.CutoffPeriodic,
    nonbondedCutoff=0.65*mm.openmm.unit.nanometer,
    removeCMMotion=True
)

print(system.usesPeriodicBoundaryConditions())

system.setDefaultPeriodicBoxVectors(*pos.getPeriodicBoxVectors())

nonbonded_force = mm.openmm.CustomNonbondedForce("(sigma/r)^6", )
nonbonded_force.addGlobalParameter('sigma', 0.2)

nonbonded_force.setCutoffDistance(0.65*mm.openmm.unit.nanometer)



for i in range(top.topology.getNumAtoms()):
    nonbonded_force.addParticle()

# nonbonded_force.createExclusionsFromBonds(
#     [[bond.atom1.index, bond.atom2.index] for bond in top.topology.bonds()],
#     bondCutoff=1
# )

# system.addForce(nonbonded_force)
# top.topology
# help(top.loadBondDefinitions)

False


In [1]:
from simulation.system_builder import SMOGSystem

sim = SMOGSystem().build_simulation()

    ****************************************************************************************    
      **** *** *** *** *** *** *** *** OpenSMOG (v1.1.1) *** *** *** *** *** *** *** ****       

           The OpenSMOG class is used to perform molecular dynamics simulations using           
                     Structure-Based Models (SBM) for biomolecular systems,                     
             and it allows for the simulation of a wide variety of potential forms.             
                      OpenSMOG uses force field files generated by SMOG 2.                      
                             OpenSMOG documentation is available at                             
                  https://opensmog.readthedocs.io and https://smog-server.org                   

                    OpenSMOG is described in: Oliveira and Contessoto et al,                    
              SMOG 2 and OpenSMOG: Extending the limits of structure-based models.              
                    Protein 

In [2]:
sim.minimizeEnergy()

In [3]:
sim.step(100000000)

ValueError: Energy is NaN.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#nan

In [4]:
import mdtraj as md
import nglview as view
traj = md.load_pdb("../data/config2/test1/test1.pdb")

In [5]:
traj

<mdtraj.Trajectory with 19930 frames, 92 atoms, 92 residues, without unitcells at 0x7fb7e6d764f0>

In [6]:
view.show_mdtraj(traj)

NGLWidget(max_frame=19929)